In [1]:
import random
import stox
from stox.imports import *
from stox.api import IB_Extractor, describe_contract
from stox import misc, yahoo
from omnibelt import tqdmd_notebook as tqdmd
from omnibelt import save_yaml
from collections import Counter
# from stox.alpha_vantage import AlphaVantageStocks

# USE the 'add-ibsym' script instead

In [2]:
symbol_table = stox.load_symbol_table()
len(symbol_table)

272

In [ ]:
yfsym = 'STMPA.DE' 

tk = yf.Ticker(yfsym)
yf_info = tk.info
print(f'{yf_info["longName"]} ')
len(yf_info)

In [3]:
ibe = IB_Extractor(True,client_id=11)

ERROR:ib_insync.wrapper: Error 321, reqId 2147483647: Error validating request.-'cB' : cause - The API interface is currently in Read-Only mode.
ERROR:ib_insync.wrapper: Error 321, reqId -1: Error validating request.-'b_' : cause - The API interface is currently in Read-Only mode.
ERROR:ib_insync.ib: open orders request timed out
ERROR:ib_insync.ib: completed orders request timed out


In [7]:
ibsym = '1COV'

currency = ''
currency = 'EUR'

ib_query = ibsym
ib_query = 'zalando'

In [8]:
search_results = [c for c in ibe.search(ib_query) if c.secType == 'STK']
print(tabulate([(i, c.symbol, c.currency, c.primaryExchange, c.conId, c.description) 
				for i, c in reversed(list(enumerate(search_results)))]))
# ibsym = ib_query.upper()

-  -------  ---  -------  ---------  ----------------------------
5  ZAL.DUM  SEK  CORPACT  500998673  ZALANDO SE - DUMMY SHARES
4  ZAL.V    SEK  VALUE    497881693  ZALANDO SE
3  ZALN     MXN  MEXI     287304372  ZALANDO SE
2  ZAL      CHF  EBS      172604484  ZALANDO SE
1  ZAL      EUR  IBIS     168731675  ZALANDO SE
0  ZLNDY    USD  PINK     193468277  ZALANDO SE - UNSPONSORED ADR
-  -------  ---  -------  ---------  ----------------------------


In [ ]:
cts = ibe.find_all_contracts(ibsym, currency=currency, secType='STK')
print(tabulate([(i, c.symbol, c.currency, c.primaryExchange, c.conId, c.description) 
				for i, c in reversed(list(enumerate(cts)))]))

In [ ]:
# ct = search_results[2]
ct = cts[0]
describe_contract(ibe, ct, name=False, industries=False, summary=False, price=False);

In [ ]:
ibq = ibsym
ibq = ct
# ibq = search_results[2]
snapshot = describe_contract(ibe, ibq, name=True, info=True, industries=False, summary=True)

In [ ]:
# yfsymbol = yfsym

yfsymbol = 'NEXI.MI'

In [ ]:
stox.add_symbol_row(symbol_table, yfsymbol, ct)

In [ ]:
# stox.save_symbol_table(symbol_table)

In [ ]:
contracts = {}

In [ ]:
root = misc.assets_root()
path = root / 'yahoo2ibkr.yml'
data = load_export(path)
len(data)

In [ ]:
reformat = {}

for row in data:
	yfsym = row['yahoo']
	del row['yahoo']
	reformat[yfsym] = row
	# reformat.append( [yfsym, row] )

# save_yaml(reformat, root / 'yahoo2ibkr.yml')

In [ ]:
len(reformat)

In [ ]:
ibe = IB_Extractor(True)

In [ ]:
bad = []
for row in tqdmd(data,key=lambda x: x['ibkr']):
	if row['ibkr'] not in contracts:
		contracts[row['ibkr']] = ibe.find_all_contracts(row['ibkr'], currency=row['currency'])
	cts = contracts[row['ibkr']]
	if len(cts) == 0 or len(set(c.conId for c in cts)) > 1:
		bad.append((row, cts))
		continue
	row['ibkr-ID'] = cts[0].conId
	row['ibkr-exchange'] = cts[0].primaryExchange
len(bad)

In [ ]:
# save_yaml(data, path);

In [ ]:
print(tabulate([[row['ibkr'], row['yahoo'], len(cts)] for row, cts in bad], headers=['IBKR', 'Yahoo', 'Num']))

In [ ]:
bad = [row for row in data if 'ibkr-ID' not in row]
bad

In [ ]:
row = bad[1]
row

In [ ]:
cts = ibe.find_all_contracts(row['ibkr'], currency=row['currency'], secType='STK')
len(cts)

In [ ]:
cts[0]

In [ ]:
row['ibkr-ID'] = cts[0].conId
row['ibkr-exchange'] = cts[0].primaryExchange

In [6]:
tkquery = 'WIE'
out = ibe.search(tkquery)
options = [c for c in out if c.secType == 'STK']
print(tabulate([(c.symbol, c.currency, c.description) for c in reversed(options)]))

ERROR:ib_insync.wrapper: Error 10159, reqId 7: Failed to request matching symbols:Error sending message to a CCP.


In [ ]:
ibtk = tkquery.upper()
info = xmltodict.parse(ibe.snapshot(ibtk))
# company_name = info['ReportSnapshot']['CoIDs']['CoID'][1]['#text']
industries = [e['#text'] for e in info['ReportSnapshot']['peerInfo']['IndustryInfo']['Industry']]
print('\n'.join(industries))
print()
summary = info['ReportSnapshot']['TextInfo']['Text'][0]['#text']
print(summary)

In [ ]:
ibe.ib.disconnect()